## Imports and Constants

In [9]:
import os
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)
os.environ["KMP_AFFINITY"] = "noverbose"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import json
import numpy as np
import pandas as pd
import tensorflow as tf
tf.autograph.set_verbosity(3)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


In [3]:
DATAPATH = 'dataset/'
MODELPATH = os.path.join(DATAPATH, 'humans/')
CLOTHINGPATH = os.path.join(DATAPATH, 'tops/')

## Load Data
### Load the humans

In [4]:
humans = pd.read_json(os.path.join(MODELPATH, 'model.json'))

In [5]:
humans

,Alex,Amanda,Anna,Anna Maria,Anneliese,Bree,Charlotte,Chesska,Chelsea,Clementine,...,Rosaline,Shadae,Shanaya,Sophia,Sophie,Stacey,Stefania,Summer,Thais,Tia
height,174.0,178.0,175.0,179.0,178.0,180.0,169.0,173.0,177.0,175.0,...,175.0,177.0,178.0,172.0,176.0,169.0,172.0,176.0,178.0,176.0
bust,81.0,86.0,84.0,84.0,84.0,99.0,88.0,89.0,84.0,86.0,...,96.0,86.0,96.0,100.0,113.0,101.0,97.0,84.0,86.5,104.0
waist,64.0,67.0,72.0,64.0,64.0,72.0,67.0,69.0,69.0,65.0,...,80.0,66.0,81.0,87.0,104.0,80.0,73.0,64.0,61.0,92.0
hips,91.0,89.0,98.0,90.0,91.0,107.0,93.0,94.0,96.0,91.0,...,107.0,91.0,107.0,109.0,132.0,98.0,103.0,96.0,92.5,111.0
url,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://images.prismic.io/birdsnest-blog/0f008...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...,https://birdsnest-images.imgix.net/system/imag...


In [7]:
humans = humans.transpose()

In [8]:
humans

,height,bust,waist,hips,url
Alex,174.0,81.0,64.0,91.0,https://birdsnest-images.imgix.net/system/imag...
Amanda,178.0,86.0,67.0,89.0,https://birdsnest-images.imgix.net/system/imag...
Anna,175.0,84.0,72.0,98.0,https://birdsnest-images.imgix.net/system/imag...
Anna Maria,179.0,84.0,64.0,90.0,https://birdsnest-images.imgix.net/system/imag...
Anneliese,178.0,84.0,64.0,91.0,https://images.prismic.io/birdsnest-blog/0f008...
...,...,...,...,...,...
Stacey,169.0,101.0,80.0,98.0,https://birdsnest-images.imgix.net/system/imag...
Stefania,172.0,97.0,73.0,103.0,https://birdsnest-images.imgix.net/system/imag...
Summer,176.0,84.0,64.0,96.0,https://birdsnest-images.imgix.net/system/imag...
Thais,178.0,86.5,61.0,92.5,https://birdsnest-images.imgix.net/system/imag...


In [13]:
# save it as a csv file
fp = os.path.join(MODELPATH, 'humans.csv')
humans.to_csv(fp)

### Load the tops
#### Load garment measurements and store in dataframe

In [66]:
# a function to do the intricate work
def parse_json(filepath):
    with open(filepath, 'r') as f:
        json_data = json.load(f)
    f.close()

    name = filepath.split('/')[-1].split('.')[0]
    
    try:
        keys = list(json_data.keys())
    except:
        return None


    data = []
    for size in keys:
        dims_data = {} # creating a dictionary for each clothing size, easier to work with
        dims_data['name'] = name
        dims_data['size'] = size

        for dim in json_data[size]:
            dims_data[dim["dimension_name"]] = dim["value"]

        data.append(dims_data)
    
    return data

In [67]:
# testing
path = 'dataset/tops/garment_measurement/avola-tunic#Sky_meta.txt'

# path = 'dataset/tops/garment_measurement/natural-textured-poncho#Deeprain_meta.txt'

parse_json(path)

In [75]:
dirpath = os.path.join(CLOTHINGPATH, 'garment_measurement')

data = []
count = 0
not_parsed = []

for file in os.listdir(dirpath):
    if file.endswith('.txt'):
        filepath = os.path.join(dirpath, file)

    data_ = parse_json(filepath)
    if data_ is None:
        count += 1
        not_parsed.append(filepath)
        continue
    
    for d in data_:
        data.append(d)

In [65]:
print(f"Number of files not parsed: {count}")

Number of files not parsed: 126


In [69]:
df

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
0,natural-textured-poncho#Deeprain_meta,S,72cm,NaN,NaN,NaN,NaN,NaN
1,natural-textured-poncho#Deeprain_meta,M,73cm,NaN,NaN,NaN,NaN,NaN
2,natural-textured-poncho#Deeprain_meta,L,74cm,NaN,NaN,NaN,NaN,NaN
3,natural-textured-poncho#Deeprain_meta,XL,75cm,NaN,NaN,NaN,NaN,NaN
4,natural-textured-poncho#Deeprain_meta,XXL,76cm,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3823,the-flutter-sleeve-top#Denim_meta,14,75cm,NaN,131cm,112cm,NaN,NaN
3824,the-flutter-sleeve-top#Denim_meta,16,78cm,NaN,137cm,118cm,NaN,NaN
3825,the-flutter-sleeve-top#Denim_meta,18,79cm,NaN,143cm,124cm,NaN,NaN
3826,the-flutter-sleeve-top#Denim_meta,20,80cm,NaN,149cm,130cm,NaN,NaN


In [76]:
df = pd.DataFrame(data)
df

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
0,natural-textured-poncho#Deeprain_meta,S,72cm,NaN,NaN,NaN,NaN,NaN
1,natural-textured-poncho#Deeprain_meta,M,73cm,NaN,NaN,NaN,NaN,NaN
2,natural-textured-poncho#Deeprain_meta,L,74cm,NaN,NaN,NaN,NaN,NaN
3,natural-textured-poncho#Deeprain_meta,XL,75cm,NaN,NaN,NaN,NaN,NaN
4,natural-textured-poncho#Deeprain_meta,XXL,76cm,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
3823,the-flutter-sleeve-top#Denim_meta,14,75cm,NaN,131cm,112cm,NaN,NaN
3824,the-flutter-sleeve-top#Denim_meta,16,78cm,NaN,137cm,118cm,NaN,NaN
3825,the-flutter-sleeve-top#Denim_meta,18,79cm,NaN,143cm,124cm,NaN,NaN
3826,the-flutter-sleeve-top#Denim_meta,20,80cm,NaN,149cm,130cm,NaN,NaN


In [77]:
df.describe()

,name,size,Length - shoulder to hem,Sleeve Length - neck to hem,Hip,Bust,Waist,Length - waist to hem
count,3828,3828,3737,3067,2953,3702,2540,6
unique,897,27,177,101,170,148,140,6
top,the-cotton-bamboo-tank#DarkNavy_meta,12,67cm,70cm,110cm,120cm,110cm,51cm
freq,9,610,190,157,79,117,91,1


In [ ]:

#todo: extract images from the given directory
#todo: extract model measurements
#todo: extract meta_descriptions

#todo?: combine the model measurements with the garment measurements to create a combined dataset
#todo?: think of processing the meta_descriptions